In [1]:
import numpy as np 
import pygame 
import math
import sys 
from copy import deepcopy 
import time 
import random 
import os
import threading

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from Ataxx.Ataxx.AtaxxGame import AtaxxGame
import Ataxx.Ataxx.AtaxxPlayers as APlayers

from Go.Go.GoGame import GoGame
import Go.Go.GoPlayers as GPlayers

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def get_ataxx_players(game):
    hp = APlayers.HumanPlayer(game)
    gp = APlayers.GreedyPlayer(game)
    A4x4p = APlayers.A4x4(game)
    A5x5p = APlayers.A5x5(game)
    A6x6p = APlayers.A6x6(game)
    AFlexp = APlayers.AFlex(game)

    return hp, gp, A4x4p, A5x5p, A6x6p, AFlexp


def get_go_players(game):
    hp = GPlayers.HumanPlayer(game)
    gp = GPlayers.GreedyPlayer(game)
    G7x7p = GPlayers.G7x7(game)
    G9x9p = GPlayers.G9x9(game)
    
    return hp, gp, G7x7p, G9x9p

In [4]:
#Game="A4x4" # "A6x6" "G7x7" "G9x9" "A5x5"
#SIZE = 5
SQUARE_SIZE = 75 

PLAYER1 = 1
PLAYER2 = -1

BLUE = (0,0,255) 
RED = (255,0,0) 
WHITE_COLOR = (255,255,255) 
BLACK_COLOR = (0,0,0) 
BACKGROUND_COLOR = (240, 196, 52)


In [5]:
import socket
import random
import time



def get_action(player, board, curPlayer, screen):
    return player.play(board, curPlayer, screen)


def get_action2(player, board, timeout=10):
    
    result = []

    def target():
        nonlocal result
        try:
            while True:
                print("getting move") 
                move =  player.play(board)
            result.append(move)
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        # Function is running longer than the timeout duration
        result.append(TimeoutError("Function execution timed out"))

    if isinstance(result[0], Exception):
        print("time is up")
        return None  # Raise any exception that occurred inside the thread

    return result[0] if result else None


    

In [6]:
#game = Ataxx(SIZE)

def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    SIZE = int(Game[-1])
    if Game[0] == "A":
        game = AtaxxGame(SIZE)
        hp, gp, A4x4p, A5x5p, A6x6p, AFlexp = get_ataxx_players(game)
        player = hp
        if player is None:
            if SIZE == 4:
                player = A4x4p
            elif SIZE == 5:
                player = A5x5p
            elif SIZE == 6:
                player = A6x6p
                
    elif Game[0] == "G":
        game = GoGame(SIZE)
        hp, gp, G7x7p, G9x9p = get_go_players(game)
        player = gp
        
        if player is None:
            if SIZE == 7:
                player = G7x7p
            elif SIZE == 9:
                player = G9x9p
        
        
    board = game.getInitBoard()
    
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    screen_size = (width, height) 
    
    
    
    if "1" in response:
        ag=1
    else:
        ag=2
    first=True
    curPlayer = 1
    
    #game.print_board()
    
    invalid_counter = 0
    
    is_pass = False
    

    screen = pygame.display.set_mode(screen_size)   
    if Game[0] == 'G':
        screen.fill(BACKGROUND_COLOR)


    pygame.display.set_caption(f'Agent {ag}')
    game.display(board, screen)

    time.sleep(10)
    
    while True:
        pygame.event.pump()
        
        if Game[0] == "A":
            if (ag == 1 or not first) and not is_pass:
                if invalid_counter >= 3:
                    is_pass = True
                    invalid_counter = 0
                    curPlayer = -curPlayer
                else:

                    if game.getGameEnded(board, curPlayer) is None:
                        action = get_action(player, game.getCanonicalForm(board, curPlayer), curPlayer, screen)
                        if action is None:
                            is_pass = True
                            invalid_counter = 0
                        else:
                            if type(action) is int:
                                new_piece = (int(action/SIZE), action%SIZE)
                                old_piece = game.get_piece_to_move(board, new_piece, curPlayer)
                                (x1, y1), (x2, y2) = old_piece, new_piece
                            else:
                                (x1, y1, x2, y2) = action

                            move_msg = f"MOVE {x1},{y1},{x2},{y2}"
                            time.sleep(1)
                            client_socket.send(move_msg.encode())
                            print("Send:",move_msg)

                    # Wait for server response
                    response = client_socket.recv(1024).decode()
                    print(f"Server Response1: {response}")
                    if "END" in response: break
                    if response == "NOT YOU":
                        print("ok")
                        curPlayer = -curPlayer
                        #continue
                    elif response == "VALID":
                        move = (x1, y1, x2, y2)
                        board, curPlayer = game.getNextState(board, curPlayer, move)
                        game.display(board, screen)
                        invalid_counter = 0
                    elif response == "INVALID":
                        invalid_counter += 1
                        continue

            first=False
            response = client_socket.recv(1024).decode()
            print(f"Server Response2: {response}")

            if response == "YOU":
                curPlayer = -curPlayer
                continue

            if "END" in response: break

            oppx1, oppy1 , oppx2, oppy2 = int(response[5]), int(response[7]), int(response[9]), int(response[11])
            board, curPlayer = game.getNextState(board, curPlayer, (oppx1, oppy1, oppx2, oppy2))

            game.display(board, screen)
            is_pass = False

        elif Game[0] == "G":
            
            if (ag == 1 or not first) and not is_pass:
                if invalid_counter >= 3:
                    is_pass = True
                    invalid_counter = 0
                    curPlayer = -curPlayer
                else:

                    if game.getGameEnded(board, curPlayer) is None:
                        action = get_action(player, game.getCanonicalForm(board, curPlayer), curPlayer, screen)
                        if action is None:
                            is_pass = True
                            invalid_counter = 0
                        else:
                            if action == game.size*game.size:
                                move = (9, 9)
                            else:
                                move = (int(action/SIZE), action%SIZE)
                            
                            #move = (int(action/SIZE), action%SIZE)
                            """
                            if type(action) is int:
                                new_piece = (int(action/SIZE), action%SIZE)
                                old_piece = game.get_piece_to_move(board, new_piece, curPlayer)
                                (x1, y1), (x2, y2) = old_piece, new_piece
                            else:
                                (x1, y1, x2, y2) = action
                            """
                            move_msg = f"MOVE {move[0]},{move[1]}"
                            time.sleep(1)
                            client_socket.send(move_msg.encode())
                            print("Send:",move_msg)

                    # Wait for server response
                    response = client_socket.recv(1024).decode()
                    print(f"Server Response1: {response}")
                    if "END" in response: break
                    if response == "NOT YOU":
                        curPlayer = -curPlayer
                        #continue
                    elif response == "VALID":
                        #move = (x1, y1, x2, y2)
                        board, curPlayer = game.getNextState(board, curPlayer, move)
                        game.display(board, screen)
                        invalid_counter = 0
                    elif response == "INVALID":
                        invalid_counter += 1
                        continue

            first=False
            response = client_socket.recv(1024).decode()
            print(f"Server Response2: {response}")

            if response == "YOU":
                curPlayer = -curPlayer
                continue

            if "END" in response: break

            oppx, oppy  = int(response[5]), int(response[7])
            board, curPlayer = game.getNextState(board, curPlayer, (oppx, oppy))

            game.display(board, screen)
            is_pass = False
                
                
                
                
                
                
                
        # Add some condition to break the loop, if necessary
        # Example: If server sends a certain message, or after a number of moves
    game_over = True

    while game_over:
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:
                pygame.quit()
                #sys.exit()

                client_socket.close()
                game_over = False
                
if __name__ == "__main__":
    pygame.init()
    game = None
    connect_to_server()

Server ResponseINIT: AG2 A4x4
Playing: A4x4


Server Response2: MOVE 0,0,0,1
0 3
0 2
Send: MOVE 0,3,0,2
Server Response1: VALID
Server Response2: MOVE 0,0,1,2
3 0
1 1
Send: MOVE 3,0,1,1
Server Response1: VALID
Server Response2: MOVE 0,3,1,3
1 1
2 3
Send: MOVE 1,1,2,3
Server Response1: VALID
Server Response2: MOVE 0,2,2,2
0 1
0 2
Send: MOVE 0,1,0,2
Server Response1: VALID
Server Response2: MOVE 2,2,1,1
1 3
3 2
Send: MOVE 1,3,3,2
Server Response1: VALID
Server Response2: MOVE 1,2,1,3
3 2
2 1
Send: MOVE 3,2,2,1
Server Response1: VALID
Server Response2: MOVE 2,3,3,1
3 3
2 3
Send: MOVE 3,3,2,3
Server Response1: VALID
Server Response2: MOVE 2,1,1,0
1 2
0 0
Send: MOVE 1,2,0,0
Server Response1: VALID
Server Response2: MOVE 0,2,1,2
1 0
2 0
Send: MOVE 1,0,2,0
Server Response1: VALID
Server Response2: MOVE 2,2,3,0
3 2
2 2
Send: MOVE 3,2,2,2
Server Response1: VALID
Server Response2: END -1 5 11
